In [ ]:
%load_ext cython
import pyrost as rst
# import speckle_tracking as st
from pyrost.bin import *
from pyrost import simulation as st_sim
from pyrost import multislice as ms_sim
import h5py
import os
import numpy as np
import pickle
import pyximport
import sys
import configparser
from multiprocessing import cpu_count
from importlib import reload
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.widgets import Slider
import matplotlib.animation as animation
plt.rcParams['animation.ffmpeg_path']= '/gpfs/cfel/user/nivanov/.conda/envs/pyrost/bin/ffmpeg'
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath, amssymb, siunitx}'
# plt.rcParams['text.usetex'] = 'True'

pyximport.install(reload_support=True, build_in_temp=False, build_dir='.pyxbld')

In [ ]:
if sys.modules.get('dev'): # Maybe sys.modules is better?
    dev = sys.modules.get('dev')
    dev = reload(dev)
else:
    import dev
print(dir(dev))

In [ ]:
scan_num = 6549
log_path = f'/gpfs/cfel/group/cxi/labs/MLL-Sigray/scan-logs/Scan_{scan_num:d}.log'
data_dir = f'/gpfs/cfel/group/cxi/labs/MLL-Sigray/scan-frames/Scan_{scan_num:d}'
data_files = sorted([os.path.join(data_dir, path) for path in os.listdir(data_dir)
                     if path.endswith('Lambda.nxs')])
wl_dict = {'Mo': 7.092917530503447e-11,
           'Cu': 1.5498024804150033e-10,
           'Rh': 6.137831605603974e-11}

In [ ]:
converter = rst.KamzikConverter()
converter = converter.read_logs(log_path)

In [ ]:
log_data = converter.cxi_get(['basis_vectors', 'log_translations'])

In [ ]:
input_file = rst.CXIStore(data_files)
# distance = 1.8
distance = 2.0
data = rst.STData(input_file, **log_data, distance=distance, wavelength=wl_dict['Mo'])

In [ ]:
data = data.load('data')
data = data.update_mask(method='range', vmax=200)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 3))
ax.imshow(data.data[1] * data.mask[1], vmax=10)
ax.set_title('Frame 0', fontsize=20)
ax.tick_params(labelsize=15)
plt.tight_layout()
plt.show()
# plt.savefig('docs/figures/sigray_frame.png', dpi=150)

In [ ]:
# crop = rst.Crop((270, 310, 430, 1250))
# crop = rst.Crop((270, 310, 330, 1170))
# crop = rst.Crop((440, 480, 790, 1200))
crop = rst.Crop((440, 470, 330, 1000))
data = data.update_transform(transform=crop)
data = data.load('data')

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 2))
ax.imshow(data.data[2], vmax=10)
ax.set_title('Frame 0', fontsize=20)
ax.tick_params(labelsize=15)
plt.tight_layout()
plt.show()
# plt.savefig('docs/figures/sigray_crop.png', dpi=150)

In [ ]:
data = data.update_mask(method='range', vmax=200).integrate_data()
good_frames = None
frames = np.arange(0, 80)
data = data.mask_frames(good_frames)
data = data.update_whitefield(frames=frames, method='robust-mean')

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(data.whitefield[0])
fig.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
n_frames = data.shape[0]

fig, ax = plt.subplots(figsize=(8, 3))
ax.imshow(data.data[data.good_frames, 0][:n_frames] / data.whitefield,
          vmin=0.0, vmax=3.0, aspect=0.75, cmap='plasma',
          extent=[0, data.shape[2] * data.x_pixel_size * 1e3,
                  (data.translations[n_frames - 1, 0] - data.translations[0, 0]) * 1e6, 0])
# ax.plot(data.whitefield[0])
ax.set_title('Ptychograph', fontsize=20)
# ax.set_yticks([0, 5, 10])
# ax.set_xlabel('Detector coordinate, \si{\milli\meter}', fontsize=20)
# ax.set_ylabel('Sample shift, \si{\micro\meter}', fontsize=20)
ax.tick_params(labelsize=20)
plt.tight_layout()
plt.show()
# plt.savefig('figures/sigray_ptychograph.pdf', dpi=300, transparent=True)

In [ ]:
defoci = np.linspace(50e-6, 500e-6, 50)
sweep_scan, extra = data.defocus_sweep(defoci, hval=5.0, return_extra=True)
defocus = defoci[np.argmax(sweep_scan)]
print(defocus)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
# ax.plot(sweep_scan)
ax.plot(extra['reference_image'][0][0])
ax.tick_params(labelsize=15)
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(defoci * 1e3, sweep_scan)
ax.set_xlabel('Defocus distance, [mm]', fontsize=15)
ax.set_title('Average gradient magnitude squared', fontsize=20)
ax.tick_params(labelsize=15)
ax.grid(True)
plt.tight_layout()
plt.show()
# plt.savefig('docs/figures/sweep_scan_sigray.png', dpi=300)

In [ ]:
data = data.update_defocus(defocus - 50e-6)
st_obj = data.get_st(test_ratio=0.1)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(st_obj.update_reference(hval=3.0).reference_image[0])
# ax.plot(np.linspace(0.1, 60.0, 150), st_obj.CV_curve(np.linspace(0.1, 60.0, 150)))
plt.show()

In [ ]:
st_res = st_obj.train_adapt(search_window=(0.0, 300.0, 0.5), h0=13.0, blur=12.0, pm_method='rsearch',
                            pm_args={'n_trias': 100, 'strides': (1, 4)})

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].plot(np.arange(st_res.reference_image.shape[1]) - st_res.ref_orig[1],
             st_res.reference_image[0])
axes[0].set_title('Reference image', fontsize=20)
axes[1].plot((st_res.pixel_map - st_obj.pixel_map)[1, 0])
axes[1].set_title('Pixel mapping', fontsize=20)
for ax in axes:
    ax.tick_params(labelsize=15)
    ax.set_xlabel('Fast axis, pixels', fontsize=15)
    ax.grid(True)
plt.tight_layout()
plt.show()
# plt.savefig('docs/figures/sigray_res.png', dpi=300)

In [ ]:
data.import_st(st_res)
fit_obj = data.get_fit(axis=1).remove_linear_term()
fit = fit_obj.fit(max_order=2)
fit['c_3']

In [ ]:
fit_obj.fit(max_order=3)['c_4'], fit_obj.fit(max_order=2)['c_3']

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].plot(fit_obj.thetas, fit_obj.theta_ab * 1e9, 'b')
axes[0].plot(fit_obj.thetas, fit_obj.model(fit['fit']) * fit_obj.ref_ap * 1e9,
             'b--', label=fr"R-PXST $c_3$ = {fit['c_3']:.4f} rad/mrad^3")
axes[0].set_title('Angular displacements, nrad', fontsize=15)

axes[1].plot(fit_obj.thetas, fit_obj.phase, 'b')
axes[1].plot(fit_obj.thetas, fit_obj.model(fit['ph_fit']), 'b--',
             label=fr"R-PXST $c_3$ ={fit['c_3']:.4f} rad/mrad^3")
axes[1].set_title('Phase, rad', fontsize=15)
for ax in axes:
    ax.legend(fontsize=10)
    ax.tick_params(labelsize=10)
    ax.set_xlabel('Scattering angles, rad', fontsize=15)
    ax.grid(True)
plt.tight_layout()
plt.show()
# plt.savefig('docs/figures/sigray_fits.png', dpi=300)

In [ ]:
phase = fit_obj.phase
wavefront = np.sqrt(data.get('whitefield')) * np.exp(1j * phase + 2.5j * np.linspace(-1.0, 1.0, phase.size)**2)
f_max = (data.x_pixel_size / data.distance / data.wavelength)**-1
psf_1d = np.fft.fftshift(np.abs(np.fft.fft(wavefront[0], 10 * phase.size))**2)
psf = psf_1d[:, None] * psf_1d[None, :]

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(1e9 * np.linspace(-f_max / 2, f_max / 2, 10 * phase.size), psf_1d)
ax.set_title(f'Scan {scan_num:d}', fontsize=20)
ax.set_xlabel('x coordinate, nm', fontsize=20)
# ax.set_ylabel('y coordinate, nm', fontsize=20)
# fig.colorbar(im, ax=ax, shrink=0.8)
ax.set_xlim(-50, 50)
# ax.set_ylim(-150, 150)
ax.tick_params(labelsize=15)
plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(9, 7.5))
im = ax.imshow(psf / psf.max(), cmap='gist_heat_r',
               extent=1e9 * np.array([-f_max / 2, f_max / 2, f_max / 2, -f_max / 2]))
ax.set_title('PSF, corrected lens', fontsize=20)
ax.set_xlabel('x coordinate, nm', fontsize=20)
ax.set_ylabel('y coordinate, nm', fontsize=20)
cbar = fig.colorbar(im, ax=ax, shrink=0.95)
cbar.ax.tick_params(labelsize=15)

ax.set_xlim(-45, 55)
ax.set_ylim(-45, 55)

ax.tick_params(labelsize=15)
fig.tight_layout()
plt.savefig('figures/psf_2f.pdf', dpi=300)
# plt.show()

In [ ]:
np.savetxt(f'results/phase_data_{scan_num:d}.txt', np.stack((fit_obj.thetas, fit_obj.phase), axis=1),
           fmt='%.6f', delimiter=',', header='2Theta Phase[rad]')

# P06 beamtime 2019

In [ ]:
!h5ls -r /gpfs/cfel/user/nivanov/p06_data/results/scan_00048/scan_st_00048.cxi

In [ ]:
data_file

In [ ]:
scan_num = 51
data_file = f"/gpfs/cfel/user/nivanov/p06_data/results/scan_{scan_num:05d}/scan_st_{scan_num:05d}.cxi"
protocol = rst.CXIProtocol.import_default()

inp_file = rst.CXIStore(data_file, 'r')

In [ ]:
with inp_file:
    data = inp_file.load_attribute('data', idxs=np.arange(10), processes=1, verbose=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 6), gridspec_kw={'width_ratios': [0.1, 1.0]})

slider = Slider(ax=ax[0], label='Frame', valmin=0, valmax=19,
                valstep=np.arange(0, 20),
                valinit=0, orientation="vertical")

im0 = ax[1].imshow(data[0], vmin=0.2, vmax=50)

def update(event):
    im0.set_data(data[slider.val])
    fig.canvas.draw_idle()

slider.on_changed(update)

fig.tight_layout()
plt.show()

In [ ]:
roi = (1100, 1565, 700, 990)
transform = rst.ComposeTransforms((rst.Crop(roi), rst.Downscale(2)))

In [ ]:
st_data = rst.STData(input_file=inp_file, transform=transform)

In [ ]:
list(inp_file)

In [ ]:
st_data = st_data.load(processes=8)

In [ ]:
st_data = st_data.update_mask(method='range-bad', vmax=1000000)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 6), gridspec_kw={'width_ratios': [0.1, 1.0]})

slider = Slider(ax=ax[0], label='Frame', valmin=0, valmax=19,
                valstep=np.arange(0, 20),
                valinit=0, orientation="vertical")

im0 = ax[1].imshow(st_data.data[0] * st_data.mask[0], vmin=0.2, vmax=200)

def update(event):
    im0.set_data(st_data.data[slider.val] * st_data.mask[slider.val])
    fig.canvas.draw_idle()

slider.on_changed(update)

fig.tight_layout()
plt.show()

In [ ]:
defoci_x = np.linspace(200e-6, 500e-6, 10)
defoci_y = np.linspace(200e-6, 500e-6, 10)
defoci_x, defoci_y = np.meshgrid(defoci_x, defoci_y)

# sweep_scan is the figure of merit of sharpness, it's maximised at the correct defocus distance
# size - the size of the local standard deviation filter,
#        the higher it is the lower spatial frequencies you take into account
# ds_x, ds_y - sampling intervals of the reference image in pixels
# hval - kernel bandwidth used in the reference image reconstruction,
#        the higher it is the blurrier the image is

sweep_scan, extra = st_data.defocus_sweep(defoci_x, defoci_y, size=50, hval=1.0,
                                          extra_args={'ds_x': 3.0, 'ds_y': 3.0},
                                          return_extra=True, verbose=True)

In [ ]:
# reshaping the R-values into an image

sweep_scan = np.array(sweep_scan).reshape(defoci_x.shape)

In [ ]:
# Finding the optimal defoci

index = np.unravel_index(np.argmax(sweep_scan), defoci_x.shape)
defocus_x, defocus_y = defoci_x[index], defoci_y[index]
print(defocus_x, defocus_y)

In [ ]:
# plotting the reference image corresponding to the optimal defocus distance

%matplotlib widget
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(extra['reference_image'][np.argmax(sweep_scan)], vmin=0.7, vmax=1.3)
fig.tight_layout()
plt.show()

In [ ]:
# parsing the found defoci to the ST data container

st_data = st_data.update_defocus(defocus_x, defocus_y)

In [ ]:
# generating a speckle tracking "reconstructer"
# ds_x, ds_y - sampling intervals of the reference image in pixels

st_obj = st_data.get_st(ds_x=2.0, ds_y=2.0)

In [ ]:
# finding the optimal reference image bandwidth
#
# kernel bandwidth used in the reference image reconstruction,
# the higher it is the blurrier the image is
#
# The cross validation error is minimized when the kernel bandwidth is "right"
# CV -> https://towardsdatascience.com/what-is-cross-validation-60c01f9d9e75

h_vals = np.linspace(0.5, 3.0, 25)
cv_vals = st_obj.CV_curve(h_vals)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(h_vals, cv_vals)
ax.set_xlabel('Kernel bandwidth', fontsize=15)
ax.set_title('Cross-validation', fontsize=20)
ax.tick_params(labelsize=10)
ax.grid(True)
fig.tight_layout()
plt.show()

In [ ]:
h0 = h_vals[np.argmin(cv_vals)]

# performing the PXST reconstruction with adaptive bandwidth
# performing the update first with high 'blur' value to supress any artifacts in pixel_map

# more info -> https://robust-speckle-tracking.readthedocs.io/en/latest/reference/rst_ref.html

st_adapt = st_obj.train_adapt(search_window=(10.0, 10.0, 0.1), h0=h0, blur=16.0, n_iter=10,
                              pm_method='rsearch', pm_args={'n_trials': 50})

In [ ]:
# Plotting the reconstructed reference image
# All the blurriness has to be corrected after the update

fig, ax = plt.subplots(figsize=(7, 6))
ax.imshow(st_adapt.reference_image, vmin=0.7, vmax=1.3)
ax.set_title('Reference image', fontsize=20)
ax.set_xlabel('horizontal axis', fontsize=15)
ax.set_ylabel('vertical axis', fontsize=15)
ax.tick_params(labelsize=15)
fig.tight_layout()
plt.show()

In [ ]:
# Plotting the reconstructed pixel_map
# You expect to see a smooth profile with following the third polynomial

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow((st_adapt.pixel_map[0] - st_obj.pixel_map[0]))
ax[1].imshow((st_adapt.pixel_map[1] - st_obj.pixel_map[1]))
fig.tight_layout()
plt.show()

In [ ]:
# Generating a phase out of st_adapt.pixel_map
# The phase is saved into st_data.phase

st_data.import_st(st_adapt)

In [ ]:
# Fitting the phase profile with st_data.get_fit
# axis - 0 for y axis and 1 for x axis

fit_obj_ss = st_data.get_fit(axis=0)
fit_ss = fit_obj_ss.fit(max_order=3)
fit_obj_fs = st_data.get_fit(axis=1)
fit_fs = fit_obj_fs.fit(max_order=3)
print(fit_fs['c_3'], fit_fs['c_4'], fit_ss['c_3'], fit_ss['c_4'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(st_data.get('phase'))
ax.set_title('Phase', fontsize=20)
ax.set_xlabel('Horizontal axis', fontsize=15)
ax.set_ylabel('Vertical axis', fontsize=15)
ax.tick_params(labelsize=15)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].plot(fit_obj_fs.pixels, fit_obj_fs.phase, label='Reconstructed profile')
axes[0].plot(fit_obj_fs.pixels, fit_obj_fs.model(fit_fs['ph_fit']), linestyle='dashed',
                 label='Polynomial fit')
axes[0].set_xlabel('Horizontal axis', fontsize=15)
axes[1].plot(fit_obj_ss.pixels, fit_obj_ss.phase, label='Reconstructed profile')
axes[1].plot(fit_obj_ss.pixels, fit_obj_ss.model(fit_ss['ph_fit']), linestyle='dashed',
             label='Polynomial fit')
axes[1].set_xlabel('Horizontal axis', fontsize=15)
for ax in axes:
    ax.set_title('Phase', fontsize=15)
    ax.tick_params(labelsize=10)
    ax.legend(fontsize=10)
    ax.grid(True)
fig.tight_layout()
plt.show()

# Multislice

In [ ]:
st_obj = data.get_st()
%time I0, n0, m0 = dev.make_reference(I_n=st_obj.data, W=st_obj.whitefield, u=st_obj.pixel_map, \
                      di=st_obj.di_pix, dj=st_obj.dj_pix, sw_y=0, \
                      sw_x=0, h=8., num_threads=st_obj.num_threads)
%time u = dev.update_pixel_map(I_n=st_obj.data, W=st_obj.whitefield, I0=I0, \
                u0=st_obj.pixel_map, di=st_obj.di_pix - n0, dj=st_obj.dj_pix - m0, \
                sw_y=0, sw_x=100, h=0.0, num_threads=st_obj.num_threads)
%time dij = dev.update_translations(I_n=st_obj.data, W=st_obj.whitefield, I0=I0, \
                u=u, di=st_obj.di_pix - n0, dj=st_obj.dj_pix - m0, \
                sw_y=0, sw_x=100, h=0.0, num_threads=st_obj.num_threads)

In [ ]:
st_obj = data.get_st()
%time _ = rst.bin.make_reference(I_n=st_obj.data, W=st_obj.whitefield, u=st_obj.pixel_map, \
                      di=st_obj.di_pix, dj=st_obj.dj_pix, sw_y=0, \
                      sw_x=0, ls=8., num_threads=st_obj.num_threads)
%time _ = rst.bin.update_pixel_map(I_n=st_obj.data, W=st_obj.whitefield, I0=I0, \
                u0=st_obj.pixel_map, di=st_obj.di_pix - n0, dj=st_obj.dj_pix - m0, \
                sw_y=0, sw_x=100, ls=0.0, num_threads=st_obj.num_threads)
%time _ = rst.bin.update_translations(I_n=st_obj.data, W=st_obj.whitefield, I0=I0, \
                u=u, di=st_obj.di_pix - n0, dj=st_obj.dj_pix - m0, \
                sw_y=0, sw_x=100, ls=0.0, num_threads=st_obj.num_threads)

In [ ]:
# Recipe simulation

en = 17.48e3
wl = ms_sim.MLL.en_to_wl / en
n_min = 700
mll_mat1 = ms_sim.Material(formula='SiC', density=2.67)
mll_mat2 = ms_sim.Material(formula='WC', density=13.87)
params = ms_sim.MSParams.import_default(x_step=1e-4, z_step=9e-3, n_min=n_min, n_max=n_min + 21710,
                                        focus=1.25e3, mll_sigma=2e-4, mll_wl=wl, wl=wl, x_min=5., x_max=45.,
                                        mll_depth=9.33, mll_mat1=mll_mat1, mll_mat2=mll_mat2)
mll = ms_sim.MLL.import_params(params)

d_n = np.loadtxt('results/mll_recipe/MLL_profiles-02.txt') * 8e-4
params.focus = 8e2
d_lyr = np.repeat(d_n / 2, 2)
z_coords = params.z_step * np.arange(params.mll_depth // params.z_step)
mll.layers = (np.cumsum(d_lyr) + 8.) - z_coords[:, None] * np.repeat(params.mll_wl / 2 / d_n, 2)
print(d_n.shape, mll.layers.shape, mll.layers.min(), mll.layers.max())
print(mll.layers.shape)

In [ ]:
# Ideal MLL simulation

en = 60.0e3
wl = ms_sim.MLL.en_to_wl / en
n_min, n_layers = 700, 20000
mll_mat1 = ms_sim.Material(formula='SiC', density=2.67)
mll_mat2 = ms_sim.Material(formula='WC', density=13.87)
params = ms_sim.MSParams.import_default(x_step=1e-4, z_step=1e-2, n_min=n_min, n_max=n_min + n_layers,
                                        focus=2.5e3, mll_sigma=2e-4, mll_wl=wl, wl=wl, x_min=5., x_max=40.,
                                        mll_depth=60.0, mll_mat1=mll_mat1, mll_mat2=mll_mat2)
mll = ms_sim.MLL.import_params(params)

ksize = 1e3
ms_prgt = ms_sim.MSPropagator(params, mll)
wf_inc = gaussian_filter(ms_prgt.wf_inc.real, ksize) + 1j * gaussian_filter(ms_prgt.wf_inc.imag, ksize)
ms_prgt = ms_prgt.update_inc_wavefront(wf_inc)

ms_prgt.beam_propagate(verbose=True)
print(mll.layers.shape, mll.layers.min(), mll.layers.max())

In [ ]:
ml_ds = (5. + np.cumsum(0.5 * mll_d * np.linspace(0.983, 0.9998, 2 * (params.n_max - params.n_min))))
ml_zs = params.z_step * np.arange(params.mll_depth // params.z_step)
layers = ml_ds - ml_zs[:, None] * np.tan(np.linspace(0, np.pi / 1.8e4, ml_ds.size))

In [ ]:
st_sim.MLL.en_to_wl / 60.00e3

In [ ]:
# Multilayer simulation

en = 60.0e3
mll_d = 10e-3
mll_z = 55.
mll_ap = 30.
wedge = 0.0
strain = 0.0

wl = st_sim.MLL.en_to_wl / en
mll_mat1 = st_sim.Material(formula='SiC', density=2.67)
mll_mat2 = st_sim.Material(formula='WC', density=13.87)
bragg = np.arcsin(0.5 * wl / mll_d)
x_mgn = 5.
params = st_sim.MSParams.import_default(x_step=1e-4, z_step=2e-2, n_min=x_mgn / mll_d,
                                        n_max=(x_mgn + mll_ap) / mll_d, mll_sigma=0.2e-4, wl=wl,
                                        x_min=0., x_max=40., mll_depth=mll_z,
                                        mll_mat1=mll_mat1, mll_mat2=mll_mat2,
                                        focus=30 / np.tan(bragg))

ml_ds = (x_mgn + np.cumsum(0.5 * mll_d * np.linspace(1.0 - strain, 1.0, 2 * (params.n_max - params.n_min))))
ml_zs = params.z_step * np.arange(params.mll_depth // params.z_step)
layers = ml_ds - ml_zs[:, None] * np.tan(np.linspace(0, wedge, ml_ds.size))
mll = st_sim.MLL(mat1_r=params.get_mat1_r(en), mat2_r=params.get_mat2_r(en),
                 sigma=params.mll_sigma, layers=layers)

ms_prgt = st_sim.MSPropagator(params, mll)

ksize = 0.33 * (ms_prgt.x_arr[1] - ms_prgt.x_arr[0])**-1 # 3 sigma = 1 um
x_lb, x_ub = x_mgn + 1., x_mgn + mll_ap - 1.
wf_inc = np.ones(ms_prgt.x_arr.shape, dtype=np.complex128)
wf_inc[(ms_prgt.x_arr < x_lb) | (ms_prgt.x_arr > x_ub)] = 0.
wf_inc = rst.bin.gaussian_filter(wf_inc.real, ksize) + \
         1j * rst.bin.gaussian_filter(wf_inc.imag, ksize)

In [ ]:
th_max = 0.15e-3
n_thetas = 250

bin_ratio = ms_prgt.size // 1000
z_arr = ms_prgt.z_arr[::int((ms_prgt.z_arr[1] - ms_prgt.z_arr[0])**-1)] # every 1 um
thetas = np.sin(bragg) + np.linspace(-th_max, th_max, n_thetas)
mll_beam = np.array([params.x_min, params.x_max])
diff_beam = mll_beam + np.tan(thetas - 2 * bragg)[:, None] * params.focus
drct_beam = mll_beam + np.tan(thetas)[:, None] * params.focus
ratio = np.ceil(2 * (np.tan(2 * bragg) + th_max) * params.focus / (params.x_max - params.x_min) + 1)
xs_foc = ratio * params.x_step * (np.arange(ms_prgt.size) - ms_prgt.size // 2) + np.mean(ms_prgt.x_arr)
xs_foc = xs_foc.reshape((bin_ratio, -1)).mean(axis=1)

In [ ]:
verbose = True
size = int((ratio * (1 + ratio) * params.x_step**2 / params.wl / params.focus)**-1)

wfts = []
for theta in tqdm(thetas, disable=not verbose,
                  bar_format='{desc} {percentage:3.0f}% {bar} Angle {n_fmt} / {total_fmt} '\
                             '[{elapsed}<{remaining}, {rate_fmt}{postfix}]'):
    ms_prgt.update_inc_wavefront.inplace_update(wf_inc * np.exp(2j * np.pi / params.wl * theta * ms_prgt.x_arr))
    ms_prgt.beam_propagate(verbose=False)
    wfts_theta = []
    for z, wft in zip(z_arr, ms_prgt.beam_profile[::int((ms_prgt.z_arr[1] - ms_prgt.z_arr[0])**-1)]):
        wft_foc = rst.bin.rsc_wp(wft, params.x_step, ratio * params.x_step,
                                 params.focus - z, params.wl)
        if size < wft.size:
            wft_foc[:(wft.size - size) // 2] = 0.0
            wft_foc[(size - wft.size) // 2:] = 0.0
        wfts_theta.append((np.abs(wft_foc)**2).reshape((bin_ratio, -1)).mean(axis=1))
    wfts.append(np.stack(wfts_theta, axis=0))
wfts = np.stack(wfts, axis=1)

In [ ]:
diff_mask = np.tile((xs_foc > diff_beam[:, 0, None]) & (xs_foc < diff_beam[:, 1, None])[None, ...],
                    (z_arr.size, 1, 1))
drct_mask = np.tile((xs_foc > drct_beam[:, 0, None]) & (xs_foc < drct_beam[:, 1, None])[None, ...],
                    (z_arr.size, 1, 1))
diff_sum = np.sum(wfts * diff_mask, axis=-1)
drct_sum = np.sum(wfts * drct_mask, axis=-1)
effs = diff_sum / (diff_sum + drct_sum)

In [ ]:
with h5py.File('results/mslice_60_keV.h5', 'r') as h5_file:
    effs = h5_file['entry/efficiencies'][()]

with h5py.File('results/mslice_10mdeg_wedge_60_keV.h5', 'r') as h5_file:
    xs_foc = h5_file['entry/xcoordinates'][()]
    wfts = h5_file['entry/wavefronts'][()]
    thetas = h5_file['entry/thetas'][()]
    z_arr = h5_file['entry/thicknesses'][()]
    effs_10_wedge = h5_file['entry/efficiencies'][()]

with h5py.File('results/mslice_20mdeg_wedge_60_keV.h5', 'r') as h5_file:
    effs_20_wedge = h5_file['entry/efficiencies'][()]

In [ ]:
with h5py.File('results/mslice_100_mllz_60_keV.h5', 'r') as h5_file:
    xs_foc = h5_file['entry/xcoordinates'][()]
    wfts = h5_file['entry/wavefronts'][()]
    thetas = h5_file['entry/thetas'][()]
    z_arr = h5_file['entry/thicknesses'][()]
    effs = h5_file['entry/efficiencies'][()]

with h5py.File('results/mslice_100_mllz_10mdeg_wedge_60_keV.h5', 'r') as h5_file:
    effs_10_wedge = h5_file['entry/efficiencies'][()]

with h5py.File('results/mslice_100_mllz_20mdeg_wedge_60_keV.h5', 'r') as h5_file:
    effs_20_wedge = h5_file['entry/efficiencies'][()]

In [ ]:
with h5py.File('results/mslice_1nm_17_keV_250_pts.h5', 'r') as h5_file:
    xs_foc = h5_file['entry/xcoordinates'][()]
    wfts = h5_file['entry/wavefronts'][()]
    thetas = h5_file['entry/thetas'][()]
    z_arr = h5_file['entry/thicknesses'][()]
    effs_1nm = h5_file['entry/efficiencies'][()]

In [ ]:
%matplotlib inline
idx = (50, 125)

fig, axes = plt.subplots(1, 1, figsize=(8, 6))
axes.plot(xs_foc, wfts[idx])
# axes.plot(xs_foc[[diff_mask[idx]]], wfts[idx][diff_mask[idx]])
# axes.plot(xs_foc[[drct_mask[idx]]], wfts[idx][drct_mask[idx]])
axes.set_title('Focal plane', fontsize=20)
axes.set_xlabel('x coordinate, um', fontsize=15)
axes.set_ylabel('Focal beam energy, a.u.', fontsize=15)
axes.tick_params(labelsize=15)
axes.grid(True)
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.animation as animation
plt.rcParams['animation.ffmpeg_path'] = '/gpfs/cfel/user/nivanov/.conda/envs/pyrost/bin/ffmpeg'

ims = []
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.set_xlabel('x coordinate, um', fontsize=15)
axes.set_ylabel('Beam absolute', fontsize=15)
axes.set_xlim(xs_foc.min(), xs_foc.max())
axes.set_ylim(0.0, wfts[50].max())
axes.tick_params(labelsize=15)

axes.grid(True)

z = 60

im_0, = axes.plot(xs_foc, wfts[z, 0], c='r',
                  label=f'Grating thickness = {z:d} um')
axes.set_title(f'Bragg offset = {np.rad2deg(thetas[0] - bragg) * 1e3:.2f} mdeg', fontsize=20)
axes.legend(fontsize=15)

def update(iternum): 
    im_0.set_data(xs_foc, wfts[z, iternum])
    axes.set_title(f'Bragg offset = {np.rad2deg(thetas[iternum] - bragg) * 1e3:.2f} mdeg', fontsize=20)
    return im_0,
    

ani = animation.FuncAnimation(fig, update, frames=thetas.size, interval=100, blit=True)
    
FFwriter = animation.FFMpegWriter(extra_args=['-vcodec', 'libx264'])
# ani.save('figures/mslice_wedged_grating_60_kev.mp4', writer=FFwriter)
plt.show()

In [ ]:
%matplotlib inline
idx = 70

fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs[idx], label='no wedge')
axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs_10_wedge[idx], label='10 mdeg wedge')
axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs_20_wedge[idx], label='20 mdeg wedge')
axes.set_title('Focal plane', fontsize=20)
axes.set_xlabel('Angle of incidence, mdeg', fontsize=15)
axes.set_ylabel('Focal beam energy, a.u.', fontsize=15)
axes.legend(fontsize=20)
axes.tick_params(labelsize=15)
axes.grid(True)
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.animation as animation
plt.rcParams['animation.ffmpeg_path'] = '/gpfs/cfel/user/nivanov/.conda/envs/pyrost/bin/ffmpeg'

ims = []
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
axes.set_xlabel('Angle of incidence, mdeg', fontsize=15)
axes.set_ylabel('Grating efficiency, [0.0 - 1.0]', fontsize=15)
axes.set_xlim(np.rad2deg(thetas - bragg).min() * 1e3, np.rad2deg(thetas - bragg).max() * 1e3)
axes.set_ylim(0.0, 1.0)
axes.tick_params(labelsize=15)

axes.grid(True)

im_0, = axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs[0], c='r', label='10 nm period')
im_1, = axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs_10_wedge[0], c='b', label='10 nm period, 10 mdeg wedge')
im_2, = axes.plot(np.rad2deg(thetas - bragg) * 1e3, effs_20_wedge[0], c='g', label='10 nm period, 20 mdeg wedge')
axes.set_title(f'Grating thickness = {z_arr[0]:.1f} um', fontsize=20)
axes.legend(fontsize=15)

def update(iternum): 
    im_0.set_data(np.rad2deg(thetas - bragg) * 1e3, effs[iternum])
    im_1.set_data(np.rad2deg(thetas - bragg) * 1e3, effs_10_wedge[iternum])
    im_2.set_data(np.rad2deg(thetas - bragg) * 1e3, effs_20_wedge[iternum])
    axes.set_title(f'Grating thickness = {z_arr[iternum]:.1f} um', fontsize=20)
    return im_0, im_1, im_2
    

ani = animation.FuncAnimation(fig, update, frames=z_arr.size, interval=200, blit=True)
    
FFwriter = animation.FFMpegWriter(extra_args=['-vcodec', 'libx264'])
# ani.save('figures/mslice_10nm_mlld.mp4', writer=FFwriter, dpi=200)
plt.show()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
axes[0].plot(ms_prgt.x_arr[::100], np.abs(ms_prgt.beam_profile[0, ::100])**2)
axes[0].set_title('Entry surface', fontsize=20)
axes[0].plot(ms_prgt.x_arr[::100], np.abs(ms_prgt.wf_inc[::100])**2)
axes[1].plot(ms_prgt.x_arr[::100], np.abs(ms_prgt.beam_profile[-1, ::100])**2)
axes[1].set_title('Exit surface', fontsize=20)
plt.show()

In [ ]:
w_pad = 100
wft = np.pad(ms_prgt.beam_profile[-1], (w_pad, w_pad))

focus_step = 4 * params.x_step
wft_focus = rsc_wp(wft, params.x_step, focus_step, params.focus - params.mll_depth, params.wl)[w_pad:-w_pad]
xfocus = focus_step * np.arange(-wft_focus.size // 2, wft_focus.size - wft_focus.size // 2)
xfocus += np.mean(ms_prgt.x_arr)

# det_dist = 2e5
# M = det_dist / params.focus * 4.
# far_step = M * params.x_step
# wft_far = fraunhofer_wp(wft, params.x_step, far_step, det_dist, params.wl)[w_pad:-w_pad]
# xfar = far_step * np.arange(-wft_far.size // 2, wft_far.size - wft_far.size // 2) + np.mean(ms_prgt.x_arr)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(xfocus, np.abs(wft_focus)**2)
plt.show()

In [ ]:
foc_step = 4 * params.x_step

wfts_foc, xs_foc = {}, {}
for z, wft in zip(ms_prgt.z_arr[::50], ms_prgt.beam_profile[::50]):

    wft_foc = rsc_wp(wft, params.x_step, foc_step, params.focus - z, params.wl)
    x_foc = foc_step * np.arange(-wft_foc.size // 2, wft_foc.size - wft_foc.size // 2) + \
            np.mean(ms_prgt.x_arr)
    
    wfts_foc[z] = wft_foc
    xs_foc[z] = x_foc

In [ ]:
foc_step = 4 * params.x_step

wfts_foc = {z: {} for z in ms_prgt.z_arr[::50]}
xs_foc = foc_step * np.arange(-ms_prgt.size // 2, ms_prgt.size - ms_prgt.size // 2) + \
         np.mean(ms_prgt.x_arr)
for bragg, ms_prgt in ms_dict.items():
    for z, wft in zip(ms_prgt.z_arr[::50], ms_prgt.beam_profile[::50]):

        wft_foc = rsc_wp(wft, params.x_step, foc_step, params.focus - z, params.wl)
        wfts_foc[z][bragg] = wft_foc

In [ ]:
sums = np.array([np.sum(np.abs(wft_foc[:wft_foc.size // 2])**2) / np.sum(np.abs(wft_foc)**2)
                 for wft_foc in wfts_foc.values()])

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 1, figsize=(10, 6))
for z in [10, 32, 59]:
    axes.plot(xs_foc[z], np.abs(wfts_foc[z])**2, label=f'MLL thickness = {z} um')
axes.legend(fontsize=15)
axes.set_xlim(44.99, 45.01)
axes.set_title('Focal plane', fontsize=20)
axes.set_xlabel('x coordinate, um', fontsize=15)
axes.set_ylabel('Intensity, a.u.', fontsize=15)
axes.tick_params(labelsize=15)
axes.grid(True)
plt.show()

In [ ]:
from pyrost.aberrations_fit import LeastSquares

x_min, x_max = 16.5, 35.
x_beam = xfocus[(xfocus > x_min) & (xfocus < x_max)]
theta = (x_beam - x_min) / params.focus

mag = np.abs(wft_focus[(xfocus > x_min) & (xfocus < x_max)])**2
phase = np.unwrap(np.angle(wft_focus[(xfocus > x_min) & (xfocus < x_max)]))

quad_fit = LeastSquares.fit(theta, phase, max_order=2)[0]
aber = phase - LeastSquares.model(quad_fit, theta, (0, x_beam.size))

ab_fit = LeastSquares.fit(theta, aber, max_order=4)[0]

In [ ]:
from pyrost.aberrations_fit import LeastSquares

# x_min, x_max = 161, 196.
x_min, x_max = 143, 171
x_beam = x_arr[(x_arr > x_min) & (x_arr < x_max)]
theta = (x_beam - x_arr.mean()) / params.focus
# theta = (x_arr[(x_arr > x_min) & (x_arr < x_max)]) / params.focus
mag = np.abs(ds_beam[:, 99][(x_arr > x_min) & (x_arr < x_max)])**2
phase = np.unwrap(np.angle(ds_beam[:, 99][(x_arr > x_min) & (x_arr < x_max)]))

quad_fit = LeastSquares.fit(theta, phase, max_order=2)[0]
aber = phase - LeastSquares.model(quad_fit, theta, (0, x_beam.size))

ab_fit = LeastSquares.fit(theta, aber, max_order=4)[0]

In [ ]:
defocus = quad_fit[-3] * params.wl / np.pi
ab_fit[-4] * 1e-9, ab_fit[-5] * 1e-12

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].plot(theta, mag, label='Multislice')
axes[0].set_title('Magnitude, a.u.', fontsize=20)
axes[1].plot(theta, phase, label='Multislice')
axes[1].plot(theta, LeastSquares.model(quad_fit, theta, (0, x_beam.size)),
             label=f'Defocus = {params.focus:.0f} um')
axes[1].set_title('Phase, rad.', fontsize=20)
axes[2].plot(theta, aber, label='Multislice')
axes[2].plot(theta, LeastSquares.model(ab_fit, theta, (0, x_beam.size)),
             label=f'$c_3$ = {ab_fit[-4] * 1e-9:.3e} rad / mrad^3\n$c_4$ = {ab_fit[-5] * 1e-12:.3e} rad / mrad^4')
axes[2].set_title('Phase aberrations, rad.', fontsize=20)
for ax in axes:
    ax.set_xlabel('x coordinate, um', fontsize=20)
    ax.tick_params(labelsize=15)
    ax.legend(fontsize=15)
plt.tight_layout()
plt.show()

In [ ]:
z_arr = np.linspace(0.1 * params.focus, 4. * params.focus, 50)
ds_beam, x_arr = ms_prgt.beam_downstream(z_arr, 1.5 * params.x_step)

In [ ]:
# from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
im1 = ax.imshow(np.abs(ds_beam[::100]), vmin=0.1, vmax=2., cmap='gist_heat_r',
                extent=[z_arr.min(), z_arr.max(), x_arr.max(), x_arr.min()])
cbar = fig.colorbar(im1, ax=ax, shrink=0.7)
cbar.ax.set_ylabel('Normalized intensity, a.u.', fontsize=20)
ax.set_title('Beam profile', fontsize=25)
ax.set_ylabel(r'x coordinate, $\mu m$', fontsize=20)
ax.set_aspect(20)
ax.tick_params(labelsize=15)
ax.set_xlabel(r'$z_1, \mu m$', fontsize=20)
ax.grid(True)
plt.tight_layout()
plt.show()

# Beamtime preparation

In [ ]:
data_obj = data_obj.crop_data((0, data.shape[1], 0, 1400))
data_obj = data_obj.update_mask(pmax=99.999)

size = 5
kernel = np.ones(int(size)) / size
mean = data_obj.whitefield.copy()
mean = fft_convolve(mean, kernel, mode='reflect', axis=0)
mean = fft_convolve(mean, kernel, mode='reflect', axis=1)
mean_sq = data_obj.whitefield**2
mean_sq = fft_convolve(mean_sq, kernel, mode='reflect', axis=0)
mean_sq = fft_convolve(mean_sq, kernel, mode='reflect', axis=1)

In [ ]:
r_image = mean_sq - mean**2
r_min = np.max(r_image[:, :50])
r_max = np.max(r_image)
r_image = np.clip(r_image, r_min, r_max) - r_min

In [ ]:
from scipy.ndimage import label, generate_binary_structure, sum_labels

s = generate_binary_structure(size, size)
labels, num_labels = label(r_image)
fbeam_lbl, db_lbl = np.argsort(sum_labels(r_image, labels, np.arange(num_labels + 1)))[-2:]
ss_arr, fs_arr = np.where(labels == fbeam_lbl)
roi = np.array([ss_arr.min(), ss_arr.max(), fs_arr.min()])

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(12, 6))
# axes.imshow(data_obj.whitefield, vmax=100)
axes.imshow(labels == fbeam_lbl, vmax=1)
plt.show()

# ST Simulation

In [ ]:
params = st_sim.STParams.import_default().replace(bar_size=5.5, bar_sigma=5., bar_atn=0.1,
                                                  bulk_atn=0.15, bar_rnd=0.7, p0=1e5, th_s=1.5e-4,
                                                  n_frames=100, defocus=120, alpha=0.00, ab_cnt=0.7,
                                                  step_size=1, step_rnd=0.8, det_dist=2e6, seed=8457259)

globals().update(**params)
print(params)

In [ ]:
n_x = params.x_wavefront_size()
n_y = params.y_wavefront_size()
nx_arr = np.arange(-n_x // 2, n_x // 2)
ny_arr = np.arange(-n_y // 2, n_y // 2)
lens_wfx, dx0 = params.lens_x_wavefront(return_step=True)
lens_wfy, dy0 = params.lens_y_wavefront(return_step=True)
print(n_x, n_y)

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(dx0 * nx_arr, np.abs(lens_wfx))
axes[0].set_xlabel('x coordinate, microns', fontsize=15)
axes[1].plot(dy0 * ny_arr, np.abs(lens_wfy))
axes[1].set_xlabel('y coordinate, microns', fontsize=15)
fig.suptitle("Wavefronts at the lens' plane", fontsize=25)
for ax in axes:
    ax.tick_params(labelsize=15)
fig.tight_layout()
plt.show()
# plt.savefig('figures/lens_wf.pdf')

In [ ]:
pad = 20000
wf_x = rsc_wp(np.pad(lens_wfx, (pad, pad)), dx0, 0.007 * dx1, params.focus, params.wl,
              num_threads=1, backend='numpy')

In [ ]:
%matplotlib widget
roi = (0 + pad + 2000, 52000 + pad + 2000)

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(np.abs(wf_x[roi[0]:roi[1]:10, None] * wf_x[None, roi[0]:roi[1]:10])**2, cmap='gist_heat_r')

fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('figures/psf_abbe.pdf', dpi=300)
plt.show()

In [ ]:
%matplotlib widget
roi = (0 + pad, 52000 + pad)
    
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(np.abs(wf_x[roi[0]:roi[1]:10, None] * wf_x[None, roi[0]:roi[1]:10])**2, cmap='gist_heat_r')

fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])

plt.savefig('figures/psf_aberrated.pdf', dpi=300)
plt.show()

In [ ]:
caustics = np.stack([rsc_wp(lens_wfx, dx0, dx1, z, params.wl, num_threads=1, backend='numpy')
                      for z in np.linspace(params.focus - params.defocus, params.focus + params.defocus, 400)], axis=1)

In [ ]:
caustics.shape

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 4))
ax.imshow(np.abs(caustics)[::10], cmap='gist_heat_r',
          extent=[-params.defocus, params.defocus, dx1 * nx_arr.min(), dx1 * nx_arr.max()])
ax.set_aspect(2e1)

fig.tight_layout()
ax.set_xticks([])
ax.set_yticks([])
plt.savefig('figures/caustics.pdf', dpi=300)
# plt.show()

In [ ]:
dx1, dy1 = dx0 * params.defocus / params.focus, dy0
z01 = params.focus + params.defocus
%time defoc_wfx = rsc_wp(lens_wfx, dx0, dx1, z01, params.wl, num_threads=1, backend='numpy')
%time defoc_wfy = rsc_wp(lens_wfy, dy0, dy1, z01, params.wl, num_threads=12, backend='numpy')

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(dx1 * nx_arr, np.abs(defoc_wfx))
axes[0].set_xlabel('x coordinate, microns', fontsize=15)
axes[1].plot(dx1 * nx_arr, np.unwrap(np.angle(defoc_wfx)))
axes[0].set_xlabel('x coordinate, microns', fontsize=15)
fig.suptitle("Wavefronts at the sample plane", fontsize=25)
for ax in axes:
    ax.tick_params(labelsize=15)
fig.tight_layout()
plt.show()
# plt.savefig('figures/sample_wf.pdf')

In [ ]:
x0, x1 = params.beam_span(params.defocus)
x1 += params.step_size * params.n_frames
bar_pos = bar_positions(x0 + params.offset, x1 - params.offset, params.bar_size, params.bar_rnd, params.seed)

In [ ]:
nx_arr.shape, params.bar_sigma / dx1

In [ ]:
smp_pos = params.sample_positions()

overlap = int(params.bar_sigma / dx1)
# nx_ext = np.arange(-(n_x + overlap) // 2, (n_x + overlap) // 2)
x_arr = dx1 * nx_arr + smp_pos[:, None]

%time b_tr = barcode_profile(x_arr=x_arr, bars=bar_pos, bulk_atn=params.bulk_atn, \
                             bar_atn=params.bar_atn, bar_sigma=0.0, \
                             num_threads=64)
b_tr = gaussian_filter(b_tr, (0.0, params.bar_sigma / dx1), num_threads=64, mode='nearest')
# b_tr = gaussian_filter(b_tr, (0.0, params.bar_sigma / dx1), num_threads=64, mode='nearest')[:, overlap // 2:-overlap // 2]
# b_tr = b_tr[:, overlap // 2:-overlap // 2]
smp_wfx = defoc_wfx * b_tr

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(np.abs(b_tr[0]))
plt.show()

In [ ]:
dx2, dy2 = params.detx_size * params.pix_size / n_x, params.dety_size * params.pix_size / n_y
%time det_wfx = fraunhofer_wp(smp_wfx, dx1, dx2, params.det_dist, params.wl, num_threads=12, backend='fftw')
%time det_wfy = fraunhofer_wp(defoc_wfy, dy1, dy2, params.det_dist, params.wl, num_threads=12, backend='fftw')
print(smp_wfx.shape, det_wfx.shape)

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(dx2 * nx_arr * 1e-3, np.abs(det_wfx[2]))
axes[0].set_xlabel('x coordinate, mm', fontsize=15)
axes[1].plot(dy2 * ny_arr * 1e-3, np.abs(det_wfy))
axes[1].set_xlabel('y coordinate, mm', fontsize=15)
fig.suptitle("Wavefronts at the detector plane", fontsize=25)
for ax in axes:
    ax.tick_params(labelsize=15)
fig.tight_layout()
plt.show()
# plt.savefig('figures/det_wf.pdf')

In [ ]:
sc_x = params.source_curve(params.defocus + params.det_dist, dx2)
sc_y = params.source_curve(params.defocus + params.det_dist, dy2)
pfx = np.sqrt(params.p0) / params.ap_x * np.abs(det_wfx)**2
pfy = np.sqrt(params.p0) / params.ap_y * np.abs(det_wfy)**2
print(pfx.shape, sc_x.shape)
%time det_ix = fft_convolve(pfx, sc_x, num_threads=12, backend='fftw')
%time det_iy = fft_convolve(pfy, sc_y, num_threads=12, backend='fftw')
# print(det_ix.shape, det_iy.shape)

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(dx2 * nx_arr * 1e-3, det_ix[62])
axes[0].set_xlabel('x coordinate, mm', fontsize=15)
axes[1].plot(dy2 * ny_arr * 1e-3, det_iy)
axes[1].set_xlabel('y coordinate, mm', fontsize=15)
fig.suptitle("Wavefronts at the detector plane", fontsize=25)
for ax in axes:
    ax.tick_params(labelsize=15)
fig.tight_layout()
plt.show()
# plt.savefig('figures/det_int.pdf')

In [ ]:
wfx = np.ones(params.detx_size)
wfy = np.ones(params.dety_size)
%time frames = make_frames(pfx=det_ix, pfy=det_iy, dx=dx2, dy=dy2, \
                           shape=(params.dety_size, params.detx_size), \
                           seed=params.seed, num_threads=64)
%time wfield = median(frames, axis=0, num_threads=64)
# frames.shape, wfield.shape

In [ ]:
%matplotlib notebook
fig, axes = plt.subplots(figsize=(12, 6))
axes.imshow(frames.sum(axis=1))
axes.set_aspect(5)
plt.show()

In [ ]:
dir_path = 'results/test'
os.makedirs(dir_path, exist_ok=True)

data = dev.make_frames(i_x=i0 * np.abs(wf1_x * det_c)**2, i_y=(np.abs(wf1_y)**2).sum() * np.ones(1),
                       sc_x=s_arr, sc_y=np.ones(1), pix_size=pix_size, noise=True)

st_params = st_sim.STParams.import_dict(**globals())
ini_parser = st_params.export_ini()
with open(os.path.join(dir_path, 'parameters.ini'), 'w') as file:
    ini_parser.write(file)
    
st_converter = st_sim.STConverter()
st_converter.save(data, st_params, dir_path, roi=roi)

In [ ]:
dir_path = 'results/test_ideal'
os.makedirs(dir_path, exist_ok=True)

data = dev.make_frames(i_x=i0 * np.abs(wf1_x * det_c)**2, i_y=(np.abs(wf1_y)**2).sum() * np.ones(1),
                       sc_x=s_arr, sc_y=np.ones(1), pix_size=pix_size, noise=False)

st_params = st_sim.STParams.import_dict(**globals())
ini_parser = st_params.export_ini()
with open(os.path.join(dir_path, 'parameters.ini'), 'w') as file:
    ini_parser.write(file)
    
st_converter = st_sim.STConverter()
st_converter.save(data, st_params, dir_path, roi=roi)

# Prototyping

In [ ]:
if sys.modules.get('dev'): # Maybe sys.modules is better?
    dev = sys.modules.get('dev')
    dev = reload(dev)
else:
    import dev
print(dir(dev))

In [ ]:
params = st_sim.STParams.import_default(bar_size=0.5, bar_sigma=0.4, bar_atn=0.319,
                                        bulk_atn=0.15, rnd_dev=0.7, p0=1.5e4, th_s=2.5e-4,
                                        n_frames=100, defocus=120, alpha=0.1, ab_cnt=0.5,
                                        step_size=0.2, step_rnd=0.8, distance=2e6, seed=8457259,
                                        det_dist=4.9e6, ap_x=20)
st_conv = st_sim.STConverter()
%time sim_obj = st_sim.STSim(params, backend='numpy')
data = st_conv.export_data(sim_obj.ptychograph(), sim_obj)

In [ ]:
mll_mat1 = ms_sim.Material(formula='SiC', density=2.67)
mll_mat2 = ms_sim.Material(formula='WC', density=13.87)
print(1 - np.exp(-2 * np.pi / params.wl *
                 mll_mat2.get_ref_index(ms_sim.Material.en_to_wl / params.wl * 1e-4).imag * 6.16))
print(1 - np.exp(-2 * np.pi / params.wl *
                 mll_mat2.get_ref_index(ms_sim.Material.en_to_wl / params.wl * 1e-4).imag * 4.16 * 0.6))

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(figsize=(10, 4))
# ax.plot(data.get('whitefield')[0])
ax.imshow(data.get('data')[:, 0], vmax=20)
ax.set_aspect(3)
plt.tight_layout()
plt.show()

In [ ]:
st_obj = data.get_st()

st_obj = st_obj.update_reference(hval=80.0)
st_res = st_obj.iter_update_gd(sw_x=20, h0=80., blur=32.0, verbose=True, n_iter=40)

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot(st_res.reference_image[0], label='Dynamic h')
axes[0].plot(st_obj.reference_image[0], label='Initial')
axes[1].plot((st_res.pixel_map - st_obj.pixel_map)[1, 0] - 
             (st_res.pixel_map - st_obj.pixel_map)[1, 0].mean(), label='Static h')
axes[0].set_title('Reference image', fontsize=20)
axes[1].set_title('Pixel map', fontsize=20)
for ax in axes:
    ax.legend(fontsize=15)
    ax.tick_params(labelsize=15)
    ax.set_xlabel('horizontal axis, pixels', fontsize=15)
    ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
data.update_phase(st_res)
fit_obj = data.get_fit()
fit_obj.defocus = np.abs(fit_obj.defocus)
fit = fit_obj.fit()
ph_fit = fit_obj.fit_phase()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].plot((fit_obj.pixels - fit_obj.pixels[0]) * fit_obj.det_ap,
             fit_obj.pixel_aberrations * fit_obj.det_ap * fit_obj.defocus * 1e9,
             label='Sigray', c='r')
axes[0].plot((fit_obj.pixels - fit_obj.pixels[0]) * fit_obj.det_ap,
             fit_obj.model(fit['fit']) * fit_obj.det_ap * fit_obj.defocus * 1e9,
             label=r'Sigray, $\alpha$ = ' + '{:.4f}'.format(fit['c_3']),
             c='r', linestyle='dashed', dashes=(8, 8))
axes[0].set_ylabel('aberrations in the sample plane, nm', fontsize=15)
axes[1].plot((fit_obj.pixels - fit_obj.pixels[0]) * fit_obj.det_ap,
             fit_obj.phase, label='Sigray', c='r')
axes[1].plot((fit_obj.pixels - fit_obj.pixels[0]) * fit_obj.det_ap, fit_obj.model(ph_fit['ph_fit']), 
             label=r'Sigray, $\alpha$ = ' + '{:.4f}'.format(fit['c_3']),
             c='r', linestyle='dashed', dashes=(8, 8))
axes[1].set_ylabel('Phase, rad', fontsize=15)
for ax in axes:
    ax.set_xlabel('2 Theta, rad', fontsize=15)
    ax.tick_params(labelsize=15)
    ax.legend(fontsize=15)
plt.tight_layout()
plt.show()